In [36]:
import pandas as pd
import geopy as gp
import numpy as np
from queue import PriorityQueue
import csv
from collections import namedtuple
from collections import defaultdict
import math
import time
from pathlib import Path  
from collections import namedtuple
from collections import defaultdict
import math
import time
from geopy.distance import geodesic as GD

In [9]:
df = pd.read_csv('Downloads/Assignment 2-2/nyc_taxi_trips/nyc_taxi_data.csv')
df.columns
df = df.drop(columns = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'Unnamed: 0', 'tpep_pickup_timestamp', 
                       'pickup_hrs', 'dropoff_hrs', 'day_week', 'tpep_dropoff_timestamp', 'duration', 'speed'])
df

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_distance
0,-73.964111,40.761398,-73.977989,40.783093,2.40
1,-73.961479,40.760410,-73.943573,40.709702,5.00
2,-73.986893,40.761726,-73.873169,40.774326,10.25
3,-73.965759,40.758114,-74.010399,40.701965,5.46
4,-73.955032,40.821857,-73.950897,40.808399,1.50
...,...,...,...,...,...
168147,-73.994392,40.751530,-73.983063,40.761108,1.10
168148,-73.960876,40.778137,-73.973915,40.764324,1.40
168149,-73.962616,40.769730,-73.975388,40.763649,1.00
168150,-74.007408,40.651272,-73.978722,40.678909,2.58


In [10]:
df_temp = df.values.tolist()
df_list = []
for x in df_temp:
        df_list.append([round(num, 4) for num in x])
tuple_list = []
for i in df_list:
    i.pop()
    tuple_list.append(tuple([i[0],i[1]]))
    tuple_list.append(tuple([i[2],i[3]]))  
df_list

[[-73.9641, 40.7614, -73.978, 40.7831],
 [-73.9615, 40.7604, -73.9436, 40.7097],
 [-73.9869, 40.7617, -73.8732, 40.7743],
 [-73.9658, 40.7581, -74.0104, 40.702],
 [-73.955, 40.8219, -73.9509, 40.8084],
 [-73.9946, 40.7188, -73.9865, 40.7345],
 [-73.9949, 40.7602, -73.9906, 40.7267],
 [-74.0044, 40.7478, -73.9997, 40.7616],
 [-73.9541, 40.7873, -73.9629, 40.7623],
 [-73.9972, 40.6844, -73.9909, 40.6928],
 [-73.9704, 40.752, -74.0053, 40.7286],
 [-74.0002, 40.7272, -73.9622, 40.7108],
 [-73.9533, 40.7832, -73.9631, 40.7626],
 [-73.9785, 40.7369, -73.9832, 40.7504],
 [-73.9945, 40.7212, -73.9428, 40.7081],
 [-74.0069, 40.7294, -73.998, 40.7089],
 [-73.9701, 40.7974, -73.9797, 40.7864],
 [-73.9919, 40.7322, -73.9835, 40.7351],
 [-73.9783, 40.7665, -73.9913, 40.75],
 [-73.9536, 40.7854, -73.9668, 40.7935],
 [-74.0077, 40.7055, -73.9475, 40.7779],
 [-73.8712, 40.774, -73.8957, 40.7126],
 [-73.9874, 40.748, -73.9709, 40.7508],
 [-73.7772, 40.6465, -73.9556, 40.7769],
 [-73.9839, 40.676, -73.9

In [11]:
def removeDuplicates(lst):
    return [t for t in (set(tuple(i) for i in lst))]

In [12]:
tuple_final = removeDuplicates(tuple_list)
tuple_final
nodes_ds = {}

In [21]:
for i in range(len(tuple_final)):
    nodes_ds[tuple_final[i]] = i
nodes_file = pd.DataFrame(nodes_ds.items()) 
# nodes_file
nodes_file.to_csv('nodes.csv', sep='\t', encoding='utf-8',index=False) 

In [22]:
edges1 = pd.concat([pd.DataFrame([nodes_ds[tuple([i[0],i[1]])], nodes_ds[tuple([i[2],i[3]])]], columns=['edge1']) for i in df_list],
          ignore_index=True)


In [23]:
edges2 = pd.concat([pd.DataFrame([nodes_ds[tuple([j[2],j[3]])], nodes_ds[tuple([j[0],j[1]])]], columns=['edge2']) for j in df_list],
          ignore_index=True)

In [24]:
dist = pd.concat([pd.DataFrame([k], columns=['distance']) for k in df['trip_distance']],
          ignore_index=True)
dist_temp = dist['distance'].repeat(2)

In [25]:
distance = pd.concat([pd.DataFrame([k], columns=['distance']) for k in dist_temp],
          ignore_index=True)

In [26]:
distance

,distance
0,2.40
1,2.40
2,5.00
3,5.00
4,10.25
...,...
336299,1.00
336300,2.58
336301,2.58
336302,30.70


In [27]:
edges = pd.concat([edges1, edges2, distance], axis = 1)
edges.to_csv('edges.csv', sep='\t', encoding='utf-8',index=False) 

In [28]:
graph = {}
for z in edges.iloc:
    if z['edge1'] in graph.keys():
        graph[int(z['edge1'])].append((int(z['edge2']), z['distance']))
    else:
        graph[int(z['edge1'])] = [(int(z['edge2']), z['distance'] )]

graph

{48302: [(28440, 2.4),
  (30201, 1.7),
  (38935, 0.82),
  (57098, 3.1),
  (39200, 1.0)],
 28440: [(48302, 2.4), (7474, 1.28), (74474, 2.08), (17353, 1.63)],
 45203: [(6674, 5.0),
  (83826, 6.4),
  (54133, 1.95),
  (136775, 0.38),
  (22436, 2.6),
  (32762, 4.8),
  (144902, 0.9),
  (96178, 3.04),
  (21203, 3.74),
  (114708, 3.6),
  (56381, 0.8),
  (68531, 1.66)],
 6674: [(45203, 5.0)],
 61331: [(60765, 10.25), (21173, 20.24), (33581, 3.7)],
 60765: [(61331, 10.25),
  (108302, 9.8),
  (28537, 9.74),
  (32787, 11.95),
  (133788, 11.2),
  (20644, 7.6),
  (72225, 2.3),
  (67262, 9.7),
  (91629, 9.4)],
 127451: [(116059, 5.46), (58900, 1.2)],
 116059: [(127451, 5.46), (17358, 5.8), (58900, 4.3)],
 86680: [(29465, 1.5)],
 29465: [(86680, 1.5)],
 140055: [(41787, 1.17),
  (4324, 4.25),
  (127838, 2.84),
  (42824, 2.89),
  (131587, 2.8)],
 41787: [(140055, 1.17), (123878, 0.4), (14845, 0.9), (38719, 2.16)],
 40084: [(75153, 3.15),
  (77737, 1.0),
  (10025, 1.28),
  (19771, 3.61),
  (20648, 0.6)]

In [29]:
start = time.time()
def search(graph, start, end):
    queue = PriorityQueue()
    queue.put((0, [start]))
    
    while not queue.empty():
        node = queue.get()
        current = node[1][len(node[1]) - 1]
        
        if end in node[1]:
            print("Path found: " + str(node[1]) + ", Cost = " + str(node[0]))
            break
        
        cost = node[0]
        for neighbor in graph[current]:
            temp = node[1][:]
            temp.append(neighbor)
            queue.put((cost + neighbor[1], temp))

In [30]:
def ucs(current_node, goal_node):
        visited = []  
        path = []
        queue = PriorityQueue()
        queue.put((0, current_node))
        
        while not queue.empty():
            item = queue.get()
            current_node =  item[1]
            cost = item[0]
            
            if current_node == goal_node:
                path.append(current_node)
                queue.queue.clear()
            else:
                if current_node in visited:
                    continue
                
                path.append(current_node)
                visited.append(current_node)
        
                for neighbour in graph[int(current_node)]:
                    queue.put((cost + neighbour[1], neighbour[0]))
        return path

In [31]:
path = ucs(140055, 4324)
print("The path will be = ",path)

The path will be =  [140055, 41787, 123878, 14845, 24976, 131587, 127838, 42824, 135056, 87416, 38719, 56447, 42771, 12424, 3880, 17834, 116038, 86136, 41190, 14302, 107437, 13485, 39578, 48937, 3917, 24436, 51672, 52178, 104838, 123911, 48460, 117424, 143192, 59067, 127284, 68996, 63050, 4324]


In [32]:
def search(graph, start, end):
    
    queue = PriorityQueue()
    queue.put((0, [start]))
    
    while not queue.empty():
        node = queue.get()
        current = node[1][len(node[1]) - 1]
        
        if end in node[1]:
            return "Path found: " + str(node[1]) + ", Cost = " + str(node[0])
            break
        
        cost = node[0]
        for neighbor in graph[current]:
            temp = node[1][:]
            temp.append(neighbor[0])
            queue.put((cost + neighbor[1] , temp))

In [33]:
cost = search(graph, 140055, 4324)
print(cost)
end = time.time()
time = end - start
print(time)

Path found: [140055, 4324], Cost = 4.25
